In [1]:
import osmnx as ox
import pandas as pd
import numpy as np

In [2]:
# longitude is x, latitude is y

G = ox.graph_from_place('San Francisco, CA, USA', network_type='drive')
nodes = ox.graph_to_gdfs(G, edges=False)

In [3]:
temp_cols = list(nodes.columns)
temp_cols[temp_cols.index('x')] = 'lng'; temp_cols[temp_cols.index('y')] = 'lat'
nodes.columns = temp_cols
nodes = nodes.sort_index()

In [4]:
# create mapping from original node number names to new node number names

node_name_map = {}; node_name_map_inv = {}
node_og = list(nodes.index)
num_nodes = len(node_og)
for i in range(num_nodes):
    node_name_map[i] = node_og[i]
    node_name_map_inv[node_og[i]] = i
nodes.index = range(nodes.shape[0])

In [5]:
df_police_stops = pd.read_csv('ca_san_francisco_2019_08_13.csv')
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lng'])]
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lat'])]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def haversine_np(long1, lat1, long2, lat2):
    """
    Calculate the great circle distance between an array of points to a single point
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    long1, lat1, long2, lat2 = map(np.radians, [long1, lat1, long2, lat2])

    dlon = long2 - long1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [10]:
from scipy.spatial import distance as dist
from time import time

coords = nodes[['lng', 'lat']].values
long1 = np.array(coords)[:, 0]
lat1 = np.array(coords)[:, 1]
batch_size = 500
police_stops = {}
m = df_police_stops.shape[0]
for i in range(0, m, batch_size):
    if i % 10000 == 0: print('%d/%d' % (i, m))
    if i + batch_size >= m:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:]
    else:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:i+batch_size]
    
    associated_nodes = np.array([haversine_np(long1, lat1, long2, lat2) for (long2, lat2) in stop_coords]).argmin(axis=1)
#     associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
    for idx in range(len(associated_nodes)):
        police_stops[associated_nodes[idx]] = police_stops.get(associated_nodes[idx], []) + [df_police_stops.iloc[i+idx].values]
        

0/903373
10000/903373
20000/903373
30000/903373
40000/903373
50000/903373
60000/903373
70000/903373
80000/903373
90000/903373
100000/903373
110000/903373
120000/903373
130000/903373
140000/903373
150000/903373
160000/903373
170000/903373
180000/903373
190000/903373
200000/903373
210000/903373
220000/903373
230000/903373
240000/903373
250000/903373
260000/903373
270000/903373
280000/903373
290000/903373
300000/903373
310000/903373
320000/903373
330000/903373
340000/903373
350000/903373
360000/903373
370000/903373
380000/903373
390000/903373
400000/903373
410000/903373
420000/903373
430000/903373
440000/903373
450000/903373
460000/903373
470000/903373
480000/903373
490000/903373
500000/903373
510000/903373
520000/903373
530000/903373
540000/903373
550000/903373
560000/903373
570000/903373
580000/903373
590000/903373
600000/903373
610000/903373
620000/903373
630000/903373
640000/903373
650000/903373
660000/903373
670000/903373
680000/903373
690000/903373
700000/903373
710000/903373
720000

In [53]:
max_edges = 0
rewards = {}
worse_reward = -100000
for node in range(num_nodes):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
    max_edges = max(max_edges, len(edges))
    # convert the edges from original names to new renamed, sorted is just to keep actions in some kind of order
    # for example the 1st action will always be the first chronological node based on naming
    edges = sorted([node_name_map_inv[edge] for edge in edges]) 
    # set reward of going to this edge equal to negative number of police stops at this node
    # if there are no police stops at the edge set reward to 0
    rewards[node] = [-len(police_stops[edge]) if edge in police_stops else 0 for edge in edges]
    
# fill terrible reward for nodes that have less total actions, encodes impossible actions essentially
for node in range(num_nodes):
    curr_actions = rewards[node]
    rewards[node] = curr_actions + [worse_reward]*(max_edges-len(curr_actions))

In [61]:
import pickle

with open('nodes_with_police_stops.pickle', 'wb') as handle:
    pickle.dump(police_stops, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('police_stop_rewards.pickle', 'wb') as handle:
    pickle.dump(rewards, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('nodes_with_police_stops.pickle', 'rb') as handle:
    police_stops = pickle.load(handle)